In [1]:
pip install --upgrade tensorflow

     |████████████████████████████████| 516.2MB 30kB/s 
  Found existing installation: tensorflow 2.2.0rc4
    Uninstalling tensorflow-2.2.0rc4:
      Successfully uninstalled tensorflow-2.2.0rc4


In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
banknote_data = pd.read_csv("https://raw.githubusercontent.com/AbhiRoy96/Banknote-Authentication-UCI-Dataset/master/bank_notes.csv")

In [5]:
banknote_data.head()

,variance,skewness,curtosis,entropy,Target
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [0]:
X =  banknote_data.drop(['Target'], axis=1)
y = banknote_data[['Target']]


In [7]:
banknote_data['Target'].value_counts()

0    762
1    610
Name: Target, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [9]:
print(X_train.shape)
print(X_test.shape)

(960, 4)
(412, 4)


In [0]:
X_train_T = X_train.copy()
X_train_T['Target'] = y_train
X_train_T = X_train_T[X_train_T['Target']==0]
X_train_T = X_train_T.drop('Target', axis=1)
X_train_T = np.asarray(X_train_T).astype(np.float32)

X_test_T = X_test.copy()
X_test_T['Target'] = y_test
X_test_T = X_test_T[X_test_T['Target']==0]
X_test_T = X_test_T.drop('Target', axis=1)
X_test_T = np.asarray(X_test_T).astype(np.float32)

In [0]:
import pandas as pd
import numpy as np
import re


from numpy import array
from tensorflow.keras.preprocessing.text import one_hot

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, LSTM, Dropout, Dense, Flatten, Input,  Embedding, Conv1D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, accuracy_score

In [12]:

epochs = 400
batch_size = 8
input_dim = X_train_T.shape[1]
encoding_dim = 2



input_layer = Input(shape=(input_dim, ))
## Encoder Layers
encoder_layer1 = Dense(encoding_dim, activation="relu")(input_layer)

decoder_output = Dense(input_dim, activation="linear")(encoder_layer1)

autoencoder = Model(inputs=input_layer, outputs=decoder_output)
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________


In [13]:
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

history = autoencoder.fit(X_train_T, X_train_T,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test_T, X_test_T),
                    verbose=1)

Epoch 1/400
67/67 [==============================] - 0s 4ms/step - loss: 27.0223 - accuracy: 0.1170 - val_loss: 26.6904 - val_accuracy: 0.1422
Epoch 2/400
67/67 [==============================] - 0s 2ms/step - loss: 23.9929 - accuracy: 0.1038 - val_loss: 24.2631 - val_accuracy: 0.1379
Epoch 3/400
67/67 [==============================] - 0s 2ms/step - loss: 21.8489 - accuracy: 0.0981 - val_loss: 22.4800 - val_accuracy: 0.1164
Epoch 4/400
67/67 [==============================] - 0s 2ms/step - loss: 20.2560 - accuracy: 0.0868 - val_loss: 21.1305 - val_accuracy: 0.1164
Epoch 5/400
67/67 [==============================] - 0s 2ms/step - loss: 19.0229 - accuracy: 0.0849 - val_loss: 20.0504 - val_accuracy: 0.1034
Epoch 6/400
67/67 [==============================] - 0s 2ms/step - loss: 18.0259 - accuracy: 0.2962 - val_loss: 19.1442 - val_accuracy: 0.6034
Epoch 7/400
67/67 [==============================] - 0s 2ms/step - loss: 17.2162 - accuracy: 0.5528 - val_loss: 18.4233 - val_accuracy: 0.6034

In [14]:
X_test_pred = autoencoder.predict(X_test)
print(X_test_pred.shape)


(412, 4)


In [15]:
mean_seq_error = np.mean(np.power(X_test - X_test_pred, 2), axis=1)
mean_seq_error

1023    3.133097
642     0.009915
1196    6.725483
31      0.892359
253     0.274953
          ...   
986     4.008020
271     1.033668
1266    3.617121
769     4.825390
634     0.598908
Length: 412, dtype: float64

In [16]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, mean_seq_error)
print('thresholds', np.mean(thresholds))
auc(false_positive_rate,  true_positive_rate)

thresholds 2.731671437730215


0.9741139846743295

In [17]:
final_threshold =  np.mean(thresholds)
accuracy_score(y_test, [1 if i > final_threshold else 0 for i in mean_seq_error])

0.9150485436893204

## Exercise 8.1 

    
    
    
### Question 1:

In an autoencoder, the number of nodes in the input layer is _____ the number of nodes in the output payer:
    
1. Smaller than 
2. Greater than 
3. Smaller than or equal to 
4. None of the above 

Answer: 4
    
    
### Question 2:

An autoencoder is a type of:
    
1. Reinforcement learning algorithm 
2. Supervised learning algorithm 
3. Unsupervised learning algorithm 
4. Semi-supervised learning algorithm 

Answer: 3
    
    
### Question 3:

Among the following, which one is an application of autoencoders?

1. Image denoising 
2. Recommendation System 
3. Dimensionality reduction 
4. All of the above 

Answer: 4





## Exercise 8.2

Using breast cancer dataset as follows, perform anomaly detection on the patients who have breast cancer. Divide this data into training and test sets. Learn the threshold for detecting anomaly on the training and use the threshold to make predictions on the complete test set. The dataset can be downloaded with the following script:

In [2]:
import pandas as pd  
from sklearn.datasets import load_breast_cancer  
cancer = load_breast_cancer()  
  
bc_data = pd.DataFrame(cancer.data, columns=cancer.feature_names)  
bc_data['Target'] = pd.Series(cancer.target)  

In [3]:
	import numpy as np  
	import matplotlib.pyplot as plt  
	from sklearn.datasets import load_breast_cancer  
	cancer = load_breast_cancer()  
	bc_data = pd.DataFrame(cancer.data, columns=cancer.feature_names)  
	bc_data['Target'] = pd.Series(cancer.target)  
	bc_data.head()  
	bc_data.shape  
	bc_data['Target'].value_counts()  
	X =  bc_data.drop(['Target'], axis=1)  
	y = bc_data[['Target']]  
	from sklearn.model_selection import train_test_split  
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)  
	print(X_train.shape)  
	print(X_test.shape)  
	X_train_T = X_train.copy()  
	X_train_T['Target'] = y_train  
	X_train_T = X_train_T[X_train_T['Target']==0]  
	X_train_T = X_train_T.drop('Target', axis=1)  
	X_train_T = np.asarray(X_train_T).astype(np.float32)  
	  
	X_test_T = X_test.copy()  
	X_test_T['Target'] = y_test  
	X_test_T = X_test_T[X_test_T['Target']==0]  
	X_test_T = X_test_T.drop('Target', axis=1)  
	X_test_T = np.asarray(X_test_T).astype(np.float32)  
	import pandas as pd  
	import numpy as np  
	import re  
	  
	  
	from numpy import array  
	from tensorflow.keras.preprocessing.text import one_hot  
	  
	from tensorflow.keras.models import Sequential  
	from tensorflow.keras.layers import Activation, LSTM, Dropout, Dense, Flatten, Input,  Embedding, Conv1D, Input  
	from tensorflow.keras.models import Model  
	from sklearn.model_selection import train_test_split  
	from tensorflow.keras import regularizers  
	from sklearn.metrics import recall_score, classification_report, auc, roc_curve, accuracy_score  
	  
	epochs = 800  
	batch_size = 2  
	input_dim = X_train_T.shape[1]  
	encoding_dim = 16  
	hidden_layers_nodes = 8  
	  
	learning_rate = 1e-3  
	  
	  
	input_layer = Input(shape=(input_dim, ))  
	## Encoder Layers  
	encoder_layer1 = Dense(encoding_dim, activation="relu")(input_layer)  
	encoder_layer2 = Dense(hidden_layers_nodes, activation="relu")(encoder_layer1)  
	  
	## Decoder Layers  
	decoder_layer1 = Dense(hidden_layers_nodes, activation="relu")(encoder_layer2)  
	decoder_layer2 = Dense(encoding_dim, activation="relu")(decoder_layer1)  
	decoder_output = Dense(input_dim, activation="linear")(decoder_layer2)  
	  
	autoencoder = Model(inputs=input_layer, outputs=decoder_output)  
	autoencoder.summary()  
	autoencoder.compile(metrics=['accuracy'],  
	                    loss='mean_squared_error',  
	                    optimizer='adam')  
	  
	history = autoencoder.fit(X_train_T, X_train_T,  
	                    epochs=epochs,  
	                    batch_size=batch_size,  
	                    shuffle=True,  
	                    validation_data=(X_test_T, X_test_T),  
	                    verbose=1)  
	X_test_pred = autoencoder.predict(X_test)  
	print(X_test_pred.shape)  
	mean_seq_error = np.mean(np.power(X_test - X_test_pred, 2), axis=1)  
	mean_seq_error  
	false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, mean_seq_error)  
	print('thresholds', np.mean(thresholds))  
	auc(false_positive_rate,  true_positive_rate)  
	final_threshold =  np.mean(thresholds)  
	accuracy_score(y_test, [1 if i > final_threshold else 0 for i in mean_seq_error])  


(398, 30)
(171, 30)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense (Dense)                (None, 16)                496       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_4 (Dense)              (None, 30)                510       
Total params: 1,358
Trainable params: 1,358
Non-trainable params: 0
_______________________________________

0.38011695906432746